In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy import signal
from scipy.optimize import curve_fit
import math
from typing import Optional
import os
import h5py

# Use filtration code developed in Filters.ipynb
import import_ipynb
from Filters import Filters, TimesteppedFilters, ButterworthFilters

importing Jupyter notebook from Filters.ipynb


In [2]:
# Unpacks signals and restores to original values in pico-Amperes
def pA_convert(path_to_fast5, filename):
    os.system('h5ls -d %s/%s.fast5/Raw/Reads/Read_981/Signal >  %s/%s.data' % (path_to_fast5, fname, path_to_fast5, fname))
    os.system('h5dump -a "/UniqueGlobalKey/channel_id/offset" %s/%s.fast5 > %s/shift.data' %(path_to_fast5,fname, path_to_fast5))
    os.system('h5dump -a "/UniqueGlobalKey/channel_id/range" %s/%s.fast5 >> %s/shift.data' %(path_to_fast5,fname, path_to_fast5))
    os.system('h5dump -a "/UniqueGlobalKey/channel_id/digitisation" %s/%s.fast5 >> %s/shift.data' %(path_to_fast5,fname, path_to_fast5))
    raw = []

    with open('%s/%s.data' % (path_to_fast5, fname), 'r') as data:
        data.readline()
        data.readline()
        line = data.readline()
        while line != '':
            splitarr = line.split()
            splitarr.pop(0)
            last = splitarr[-1]
            for current in range(len(splitarr)):
                splitarr[current] = splitarr[current][:-1]
            raw.extend(splitarr)
            line = data.readline()
        raw[-1] = last
    
    #get offset and scale
    with open('%s/shift.data' % (path_to_fast5), 'r') as shift:
        for a in range(5):
            shift.readline()
        offsetLine = shift.readline()
        offsetArr = offsetLine.split()
        offsetArr.pop(0)
        offset = float(offsetArr[0])
    
        for b in range(8):
            shift.readline()
        rangeLine = shift.readline()
        rangeArr = rangeLine.split()
        rangeArr.pop(0)
        rang = float(rangeArr[0])
    
        for c in range(8):
            shift.readline()
        digLine = shift.readline()
        digArr = digLine.split()
        digArr.pop(0)
        dig = float(digArr[0])
    print("Offset:", offset, ", Range:", rang, ", Digitisation:", dig)

    os.system('rm %s/%s.data' % (path_to_fast5, fname))
    os.system('rm %s/shift.data' % (path_to_fast5))

    scale = rang/dig
    raw = np.array(raw, dtype=np.float32)

    signal = (raw + offset)*scale
    return signal

In [3]:
# Restore noisy and truth signal arrays to pico-Amperes
path_to_fast5 = '/Users/aaronphilip/ScienceFair/projects/NanoporeSequencingFiltering/DeepSimulator/artificial_human_chr22_DeepSimu/fast5'
fname = 'signal_0_30d8bd50-3aae-4eab-95da-2083e9df0631'
noisySignal = pA_convert(path_to_fast5, fname)

Offset: 37.0 , Range: 1368.36 , Digitisation: 8192.0


In [4]:
#print(noisySignal.size)
#cutLen = 1000
#ncuts = (int) (len(raw)/cutLen)
#segmentArray = np.empty(ncuts) #tells where to 'cut' the raw data; to be refined later (cwDTW_nano)
#print(segmentArray.size)

#for length in range(segmentArray.size):
#    segmentArray[length] = cutLen